# Glands Segmentator #


In this file, I use two models that I created to successfully perform gland segmentation. This is a notebook that processes big images, which are divided into tiles. The first model is used to detect the regions of interest (glands), and the second model is used to segment the glands in those regions. 

# Procesing of big images #

Imports:

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import tifffile
import math
import random

Loading models:

In [4]:
import unet_core.unet_interface

In [5]:
import importlib.util
spec = importlib.util.find_spec("unet_core.unet_interface")

In [5]:
detectionModel = YOLO("../model/saved_models/Glands_Finder_Augumented_Data_best.pt")
from unet_core.unet_interface import UNET
modelUNET = UNET(img_height=50, img_width=50)
modelUNET.set_model(in_channels=3, out_channels=1,
                    name="C:/Users/stszy/miniconda3/Lib/site-packages/unet_core/GlandsFinder")


Chosen model successfully found
=> Loading checkpoint


 Helper function:

In [6]:
def tile_image(image, tile_size, overlap):
    tiles = []
    positions = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    for y in range(0, h, stride):
        for x in range(0, w, stride):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y))
    return tiles, positions


def detect_on_tiles(tiles, positions, model):
    all_detections = []
    for tile, (x_offset, y_offset) in zip(tiles, positions):
        results = model(tile, conf=0.9)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1 += x_offset
            x2 += x_offset
            y1 += y_offset
            y2 += y_offset
            conf = float(box.conf)
            cls = int(box.cls)
            all_detections.append((x1, y1, x2, y2, conf, cls))
    return all_detections


def draw_detections(image, detections, class_names):
    annotated = image.copy()
    for x1, y1, x2, y2, conf, cls in detections:
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"{class_names[cls]} {conf:.2f}"
        cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return annotated

def merge_overlapping_boxes(
        detections,
        merge_distance=80,
        size_similarity_thresh=0.3
):
    merged = []
    used = [False] * len(detections)

    for i, det in enumerate(detections):
        if used[i]:
            continue

        x1, y1, x2, y2, conf, cls = det
        w1 = x2 - x1
        h1 = y2 - y1
        cx1 = (x1 + x2) / 2
        cy1 = (y1 + y2) / 2
        group = [(x1, y1, x2, y2, conf)]
        used[i] = True

        for j in range(i + 1, len(detections)):
            if used[j]:
                continue

            x1_b, y1_b, x2_b, y2_b, conf_b, cls_b = detections[j]
            w2 = x2_b - x1_b
            h2 = y2_b - y1_b
            cx2 = (x1_b + x2_b) / 2
            cy2 = (y1_b + y2_b) / 2

            dist = math.hypot(cx1 - cx2, cy1 - cy2)
            if dist > merge_distance:
                continue

            width_similar = abs(w1 - w2) / max(w1, w2) <= size_similarity_thresh
            height_similar = abs(h1 - h2) / max(h1, h2) <= size_similarity_thresh
            if not (width_similar or height_similar):
                continue

            group.append((x1_b, y1_b, x2_b, y2_b, conf_b))
            used[j] = True

        xs1, ys1, xs2, ys2, confs = zip(*group)
        merged_box = (
            min(xs1),
            min(ys1),
            max(xs2),
            max(ys2),
            sum(confs) / len(confs),
            cls
        )
        merged.append(merged_box)

    return merged

def draw_detections_varied_colors(image, detections, class_names, thickness=2, font_scale=0.5):
    annotated = image.copy()
    random.seed(42)

    for idx, det in enumerate(detections):
        x1, y1, x2, y2, conf, cls_id = det

        color = tuple(int(c) for c in np.random.randint(0, 256, size=3))

        label = f"{class_names[int(cls_id)]} {conf:.2f}"

        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        text_y = int(y1) - 5 if y1 > 10 else int(y1) + 15
        cv2.putText(
            annotated,
            label,
            (int(x1), text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            color,
            thickness=1,
            lineType=cv2.LINE_AA
        )

    return annotated

# Detecting on tiles

In [7]:
image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048
overlap = 1024
tiles, positions = tile_image(image, tile_size, overlap)

detections = detect_on_tiles(tiles, positions, detectionModel)

detections = merge_overlapping_boxes(detections, merge_distance=80)

class_names = detectionModel.names

annotated = draw_detections(image, detections, class_names)

cv2.imwrite("wynik_z_detections.png", annotated)

annotated = draw_detections_varied_colors(image, detections, class_names)
cv2.imwrite("wynik_z_detections_varied_colors.png", annotated)



0: 640x640 (no detections), 139.0ms
Speed: 4.6ms preprocess, 139.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 73.4ms
Speed: 3.9ms preprocess, 73.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 66.9ms
Speed: 3.2ms preprocess, 66.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 70.9ms
Speed: 3.2ms preprocess, 70.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 66.3ms
Speed: 3.3ms preprocess, 66.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 65.6ms
Speed: 3.4ms preprocess, 65.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 65.6ms
Speed: 3.2ms preprocess, 65.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 62.6ms
Speed: 3.5ms preprocess, 62.6ms

True

In [8]:
import numpy as np


def tile_image(image, tile_size, overlap):
    tiles = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    ys = list(range(0, h, stride))
    xs = list(range(0, w, stride))

    n_tiles_y = len(ys)
    n_tiles_x = len(xs)

    positions_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions_matrix[i, j] = (x, y)

    return tiles, positions_matrix


In [9]:
image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 4098
overlap = 1024

class_names = detectionModel.names
tiles, positions_matrix = tile_image(image, tile_size, overlap)

In [10]:
print("Number of tiles:", len(tiles))
print("Positions matrix shape:", positions_matrix.shape)
print(positions_matrix)

Number of tiles: 42
Positions matrix shape: (6, 7)
[[(0, 0) (3074, 0) (6148, 0) (9222, 0) (12296, 0) (15370, 0) (18444, 0)]
 [(0, 3074) (3074, 3074) (6148, 3074) (9222, 3074) (12296, 3074) (15370, 3074) (18444, 3074)]
 [(0, 6148) (3074, 6148) (6148, 6148) (9222, 6148) (12296, 6148) (15370, 6148) (18444, 6148)]
 [(0, 9222) (3074, 9222) (6148, 9222) (9222, 9222) (12296, 9222) (15370, 9222) (18444, 9222)]
 [(0, 12296) (3074, 12296) (6148, 12296) (9222, 12296) (12296, 12296) (15370, 12296) (18444, 12296)]
 [(0, 15370) (3074, 15370) (6148, 15370) (9222, 15370) (12296, 15370) (15370, 15370) (18444, 15370)]]


# Segmenting on tiles

In [11]:
def tile_image_with_detection(image, tile_size, overlap, model):
    tiles = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    ys = list(range(0, h, stride))
    xs = list(range(0, w, stride))

    n_tiles_y = len(ys)
    n_tiles_x = len(xs)

    detection_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            results = model(tile, conf=0.9)[0]
            list_of_boxes = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1 += x
                x2 += x
                y1 += y
                y2 += y
                conf = float(box.conf)
                cls = int(box.cls)
                list_of_boxes.append((x1, y1, x2, y2, conf, cls))

            detection_matrix[i, j] = (x, y, list_of_boxes)

    return tiles, detection_matrix


In [12]:
image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048
overlap = 1024

class_names = detectionModel.names
tiles, positions_matrix = tile_image_with_detection(image, tile_size, overlap, detectionModel)


0: 640x640 (no detections), 90.8ms
Speed: 3.6ms preprocess, 90.8ms inference, 34.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 69.0ms
Speed: 3.4ms preprocess, 69.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 63.4ms
Speed: 3.1ms preprocess, 63.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 59.5ms
Speed: 3.0ms preprocess, 59.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 63.5ms
Speed: 3.0ms preprocess, 63.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 58.9ms
Speed: 3.0ms preprocess, 58.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 74.7ms
Speed: 4.2ms preprocess, 74.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 64.2ms
Speed: 3.4ms preprocess, 64.2ms 

In [13]:
print("Number of tiles:", len(tiles))
print("Positions matrix shape:", positions_matrix.shape)
print(positions_matrix)

Number of tiles: 304
Positions matrix shape: (16, 19)
[[(0, 0, []) (1024, 0, []) (2048, 0, []) (3072, 0, []) (4096, 0, []) (5120, 0, []) (6144, 0, []) (7168, 0, []) (8192, 0, [])
  (9216, 0, [(np.float32(9958.877), np.float32(1552.4283), np.float32(10544.705), np.float32(2036.5139), 0.9550598859786987, 0), (np.float32(9675.885), np.float32(1169.074), np.float32(10241.976), np.float32(1621.9808), 0.9130753874778748, 0), (np.float32(10323.123), np.float32(1476.7266), np.float32(10739.624), np.float32(1878.3828), 0.9123066067695618, 0)])
  (10240, 0, [(np.float32(10302.937), np.float32(1469.0115), np.float32(10739.961), np.float32(1878.4231), 0.9474421143531799, 0)]) (11264, 0, []) (12288, 0, []) (13312, 0, []) (14336, 0, []) (15360, 0, []) (16384, 0, []) (17408, 0, []) (18432, 0, [])]
 [(0, 1024, []) (1024, 1024, []) (2048, 1024, []) (3072, 1024, []) (4096, 1024, []) (5120, 1024, [])
  (6144, 1024, [(np.float32(7873.7373), np.float32(2751.3354), np.float32(8192.0), np.float32(3065.2505),

In [14]:
def draw_detections_from_matrix(image, detection_matrix, class_names):
    annotated = image.copy()
    for i in range(detection_matrix.shape[0]):
        for j in range(detection_matrix.shape[1]):
            x, y, detections = detection_matrix[i, j]

            if detections is not None:
                for det in detections:
                    x1, y1, x2, y2, conf, cls = det
                    cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    label = f"{class_names[cls]} {conf:.2f}"
                    cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return annotated



In [15]:
annotated = draw_detections_from_matrix(image, positions_matrix, class_names)

cv2.imwrite("matrix.png", annotated)


True

In [16]:
def sort_boxes_by_confidence(detection_matrix):
    sorted_matrix = np.empty_like(detection_matrix, dtype=object)
    for i in range(detection_matrix.shape[0]):
        for j in range(detection_matrix.shape[1]):
            x, y, detections = detection_matrix[i, j]
            if detections is not None:
                sorted_detections = sorted(detections, key=lambda det: det[4], reverse=True)
                sorted_matrix[i, j] = (x, y, sorted_detections)
            else:
                sorted_matrix[i, j] = (x, y, None)
    return sorted_matrix

In [17]:
sorted_positions_matrix = sort_boxes_by_confidence(positions_matrix)

In [18]:
print(sorted_positions_matrix)

[[(0, 0, []) (1024, 0, []) (2048, 0, []) (3072, 0, []) (4096, 0, []) (5120, 0, []) (6144, 0, []) (7168, 0, []) (8192, 0, [])
  (9216, 0, [(np.float32(9958.877), np.float32(1552.4283), np.float32(10544.705), np.float32(2036.5139), 0.9550598859786987, 0), (np.float32(9675.885), np.float32(1169.074), np.float32(10241.976), np.float32(1621.9808), 0.9130753874778748, 0), (np.float32(10323.123), np.float32(1476.7266), np.float32(10739.624), np.float32(1878.3828), 0.9123066067695618, 0)])
  (10240, 0, [(np.float32(10302.937), np.float32(1469.0115), np.float32(10739.961), np.float32(1878.4231), 0.9474421143531799, 0)]) (11264, 0, []) (12288, 0, []) (13312, 0, []) (14336, 0, []) (15360, 0, []) (16384, 0, []) (17408, 0, []) (18432, 0, [])]
 [(0, 1024, []) (1024, 1024, []) (2048, 1024, []) (3072, 1024, []) (4096, 1024, []) (5120, 1024, [])
  (6144, 1024, [(np.float32(7873.7373), np.float32(2751.3354), np.float32(8192.0), np.float32(3065.2505), 0.9165497422218323, 0), (np.float32(7924.954), np.flo

In [19]:
def compute_iou(boxA, boxB):
    # box: (x1, y1, x2, y2, conf, cls)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0
    overlap_ratio = interArea / float(min(boxAArea, boxBArea)) if min(boxAArea, boxBArea) > 0 else 0
    return iou, overlap_ratio

def dimensions_similar(boxA, boxB, dim_thresh=0.3):

    widthA = boxA[2] - boxA[0]
    heightA = boxA[3] - boxA[1]
    widthB = boxB[2] - boxB[0]
    heightB = boxB[3] - boxB[1]

    width_similar = abs(widthA - widthB) / max(widthA, widthB) < dim_thresh
    height_similar = abs(heightA - heightB) / max(heightA, heightB) < dim_thresh

    return width_similar or height_similar

def assign_box_ids(grid, iou_thresh=0.4, overlap_thresh=0.60, dim_thresh=0.25):

    processed = []
    next_id = 1
    result = []
    total_detections = 0

    for row in grid:
        result_row = []
        for x_cell, y_cell, dets in row:
            if not dets:
                result_row.append((x_cell, y_cell, []))
                continue

            dets_sorted = sorted(dets, key=lambda b: b[4], reverse=True)
            dets_with_id = []
            for box in dets_sorted:
                total_detections += 1
                assigned = None
                for prev in processed:
                    iou, overlap = compute_iou(box, prev['box'])
                    if (iou > iou_thresh or overlap > overlap_thresh) and dimensions_similar(box, prev['box'], dim_thresh):
                        assigned = prev['id']
                        break

                if assigned is None:
                    assigned = next_id
                    next_id += 1

                processed.append({'box': box, 'id': assigned})
                dets_with_id.append((*box, assigned))

            result_row.append((x_cell, y_cell, dets_with_id))
        result.append(result_row)

    print(f"Total detections: {total_detections}")
    print(f"Unique IDs assigned: {next_id - 1}")
    return result


In [20]:
matrix_ready_for_segmentation=assign_box_ids(sorted_positions_matrix)

Total detections: 2087
Unique IDs assigned: 603


In [21]:
def build_unique_detections_matrix(annotated_grid):
    unique_boxes = {}

    for row in annotated_grid:
        for x_cell, y_cell, dets in row:
            for box in dets:
                x1, y1, x2, y2, conf, cls, box_id = box
                if box_id not in unique_boxes:
                    unique_boxes[box_id] = [x1, y1, x2, y2, conf, cls]
                else:
                    prev = unique_boxes[box_id]
                    merged = [
                        min(prev[0], x1),
                        min(prev[1], y1),
                        max(prev[2], x2),
                        max(prev[3], y2),
                        max(prev[4], conf),
                        cls 
                    ]
                    unique_boxes[box_id] = merged

    # Convert to list format
    return [(*vals, box_id) for box_id, vals in unique_boxes.items()]

In [22]:
unique_detections = build_unique_detections_matrix(matrix_ready_for_segmentation)

In [23]:
def draw_detections(image, detections, class_names):
    annotated = image.copy()
    for x1, y1, x2, y2, conf, cls, box_id in detections:
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"{class_names[int(cls)]} {conf:.2f} ID:{box_id}"
        cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return annotated

In [32]:
annotated = draw_detections(image, unique_detections, class_names)
cv2.imwrite("merged.png", annotated)


True

In [25]:
print(f"first: ", unique_detections[0])

first:  (np.float32(9956.611), np.float32(1552.4125), np.float32(10544.705), np.float32(2036.5139), 0.9550598859786987, 0, 1)


In [26]:
import numpy as np
import cv2
from PIL import Image

def segment_on_tiles(image, unique_detections, modelUNET):

    segmentations = []

    for detection in unique_detections:
        x1, y1, x2, y2 = map(int, detection[:4])

        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(image.shape[1], x2)
        y2 = min(image.shape[0], y2)

        tile = image[y1:y2, x1:x2]
        if tile.size == 0:
            continue

        tile_rgb = cv2.cvtColor(tile, cv2.COLOR_BGR2RGB)
        tile_pil = Image.fromarray(tile_rgb)

        local_mask = modelUNET.find_points(tile_pil)

        
        global_mask = [
            [ (px + x1, py + y1) for (px, py) in contour ]
            for contour in local_mask
        ]

        segmentations.append({
            'bbox': (x1, y1, x2, y2),
            'mask': global_mask,
            'id': detection[6] if len(detection) > 6 else None,
        })
        print(f"Segmented tile at ({x1}, {y1}) to ({x2}, {y2}); found {len(global_mask)} contour(s)")

    return segmentations


In [27]:
segmentations=segment_on_tiles(image, unique_detections, modelUNET)
print("lokks like : segmentations: ", segmentations[0])

Segmented tile at (9956, 1552) to (10544, 2036); found 0 contour(s)
Segmented tile at (9669, 1144) to (10245, 1623); found 0 contour(s)
Segmented tile at (10302, 1469) to (10741, 1879); found 0 contour(s)
Segmented tile at (7870, 2750) to (8304, 3073); found 1 contour(s)
Segmented tile at (7924, 2583) to (8152, 2757); found 1 contour(s)
Segmented tile at (8497, 2184) to (9096, 2811); found 0 contour(s)
Segmented tile at (8265, 2639) to (8553, 2907); found 0 contour(s)
Segmented tile at (8408, 2444) to (9040, 2998); found 0 contour(s)
Segmented tile at (8927, 1976) to (9513, 2770); found 0 contour(s)
Segmented tile at (9032, 2582) to (9290, 2848); found 0 contour(s)
Segmented tile at (9748, 1880) to (10290, 2333); found 1 contour(s)
Segmented tile at (9705, 2092) to (10111, 2456); found 0 contour(s)
Segmented tile at (7670, 2942) to (7978, 3163); found 0 contour(s)
Segmented tile at (7755, 3666) to (8033, 3819); found 0 contour(s)
Segmented tile at (7729, 3849) to (8074, 4098); found 0 

In [28]:
print("lokks like : segmentations: ", segmentations)


lokks like : segmentations:  [{'bbox': (9956, 1552, 10544, 2036), 'mask': [], 'id': 1}, {'bbox': (9669, 1144, 10245, 1623), 'mask': [], 'id': 2}, {'bbox': (10302, 1469, 10741, 1879), 'mask': [], 'id': 3}, {'bbox': (7870, 2750, 8304, 3073), 'mask': [[(7870, 2750)]], 'id': 4}, {'bbox': (7924, 2583, 8152, 2757), 'mask': [[(8147, 2583)]], 'id': 5}, {'bbox': (8497, 2184, 9096, 2811), 'mask': [], 'id': 6}, {'bbox': (8265, 2639, 8553, 2907), 'mask': [], 'id': 7}, {'bbox': (8408, 2444, 9040, 2998), 'mask': [], 'id': 8}, {'bbox': (8927, 1976, 9513, 2770), 'mask': [], 'id': 9}, {'bbox': (9032, 2582, 9290, 2848), 'mask': [], 'id': 10}, {'bbox': (9748, 1880, 10290, 2333), 'mask': [[(10279, 1880)]], 'id': 11}, {'bbox': (9705, 2092, 10111, 2456), 'mask': [], 'id': 12}, {'bbox': (7670, 2942, 7978, 3163), 'mask': [], 'id': 13}, {'bbox': (7755, 3666, 8033, 3819), 'mask': [], 'id': 14}, {'bbox': (7729, 3849, 8074, 4098), 'mask': [], 'id': 15}, {'bbox': (7563, 3500, 8036, 3657), 'mask': [], 'id': 16}, {'

# Drawing segmentations on the image

In [29]:
def apply_segmentations_to_image(image, segmentations, color=(0, 255, 0), thickness=2):
    img_with_segmentations = image.copy()

    for i, seg in enumerate(segmentations):
        mask = seg.get('mask', [])
        if not mask:
            print(f"Segment {i}: empty mask, skipping")
            continue

        if isinstance(mask[0], (list, tuple)) and all(isinstance(p, (list, tuple)) and len(p) == 2 for p in mask[0]):
            pts = mask[0]
        else:
            pts = mask

        if not pts:
            print(f"Segment {i}: unpacked mask is empty, skipping")
            continue

        first_point = pts[0]
        if isinstance(first_point, dict) and 'x' in first_point and 'y' in first_point:
            pts = [(point["x"], point["y"]) for point in pts]
        elif isinstance(first_point, (list, tuple)) and len(first_point) == 2:
            pts = [(int(x), int(y)) for x, y in pts]
        else:
            raise ValueError(f"Unknown mask point format for segment {i}: {pts}")

        if len(pts) == 1:
            cv2.circle(img_with_segmentations, pts[0], radius=5, color=color, thickness=-1)
        else:
            pts = np.array(pts, dtype=np.int32).reshape((-1, 1, 2))
            cv2.polylines(img_with_segmentations, [pts], isClosed=True, color=color, thickness=thickness)

    return img_with_segmentations

In [30]:
segmented_image = apply_segmentations_to_image(image, segmentations)

cv2.imwrite("segmented.png", segmented_image)


Segment 0: pusta maska, pomijam
Segment 1: pusta maska, pomijam
Segment 2: pusta maska, pomijam
Segment 5: pusta maska, pomijam
Segment 6: pusta maska, pomijam
Segment 7: pusta maska, pomijam
Segment 8: pusta maska, pomijam
Segment 9: pusta maska, pomijam
Segment 11: pusta maska, pomijam
Segment 12: pusta maska, pomijam
Segment 13: pusta maska, pomijam
Segment 14: pusta maska, pomijam
Segment 15: pusta maska, pomijam
Segment 16: pusta maska, pomijam
Segment 17: pusta maska, pomijam
Segment 18: pusta maska, pomijam
Segment 19: pusta maska, pomijam
Segment 20: pusta maska, pomijam
Segment 21: pusta maska, pomijam
Segment 22: pusta maska, pomijam
Segment 23: pusta maska, pomijam
Segment 24: pusta maska, pomijam
Segment 26: pusta maska, pomijam
Segment 27: pusta maska, pomijam
Segment 29: pusta maska, pomijam
Segment 30: pusta maska, pomijam
Segment 31: pusta maska, pomijam
Segment 33: pusta maska, pomijam
Segment 34: pusta maska, pomijam
Segment 35: pusta maska, pomijam
Segment 36: pusta 

True